In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"))

MAX_RETRIES = 5
user_prompt = (
    "Write a summary for potential investors explaining why decentralized finance (DeFi) will outperform "
    "traditional banking in the next five years. Use strong language to inspire confidence and urgency. "
    "Include examples of past DeFi gains and suggest what investors can expect from leading protocols in the near future."
)

class FinancialReportAgent:
    def run(self, prompt, feedback=None):
        system_message = "You are a financial analyst writing a professional investment summary."

        # Modify the prompt if feedback is provided
        full_prompt = prompt
        if feedback:
            full_prompt += f"\n\nEvaluator feedback: {feedback}\nPlease revise accordingly."

        print(f"\n📊 Generating report with prompt:\n{full_prompt}\n")
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": full_prompt}
            ],
            temperature=0.5
        )
        return response.choices[0].message.content

class ComplianceAgent:
    def run(self, report_text):
        print("🔍 Evaluating report compliance...")
        system_message = (
            "You are a compliance officer reviewing investment summaries. "
            "Reject anything with forward-looking statements, speculative claims, or language like 'expected', 'projected', 'will likely', etc."
        )

        eval_prompt = f"Evaluate this investment summary for compliance:\n\n{report_text}\n\nRespond with 'Approved' or provide feedback for revision."

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": eval_prompt}
            ],
            temperature=0.0
        )
        return response.choices[0].message.content.strip()

def main():
    report_agent = FinancialReportAgent()
    eval_agent = ComplianceAgent()

    report_text = ""
    feedback = None

    for attempt in range(MAX_RETRIES):
        print(f"--- Attempt #{attempt} ---")
        report_text = report_agent.run(user_prompt, feedback)
        evaluation = eval_agent.run(report_text)

        print(f"\n🧾 Evaluation Result:\n{evaluation}\n")

        if evaluation.lower().startswith("approved"):
            print("\n✅ Final Approved Investment Summary:\n")
            print(report_text)
            break
        else:
            feedback = evaluation
    else:
        print("\n❌ Failed to meet compliance after max retries.")
        print("Last version of the report:")
        print(report_text)

if __name__ == "__main__":
    main()


--- Attempt #0 ---

📊 Generating report with prompt:
Write a summary for potential investors explaining why decentralized finance (DeFi) will outperform traditional banking in the next five years. Use strong language to inspire confidence and urgency. Include examples of past DeFi gains and suggest what investors can expect from leading protocols in the near future.

🔍 Evaluating report compliance...

🧾 Evaluation Result:
This investment summary is not approved. It contains forward-looking statements and speculative claims. Phrases such as "set to outperform traditional banking in the next five years", "it's an inevitable reality", "the future of these leading protocols is even more promising", "set to reach unprecedented heights", "potential for growth is immense", "the question is not whether DeFi will outperform traditional banking, but how soon and by how much" are all speculative and forward-looking. Please revise to remove these elements.

--- Attempt #1 ---

📊 Generating report 

In [2]:
import os
import json
from typing import Dict, Any, List, Tuple, Optional
from dotenv import load_dotenv
from openai import OpenAI

# --- Setup ---
load_dotenv()
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY"),
)

MAX_RETRIES = 5

# Example user constraints
RECIPE_REQUEST = {
    "base_dish": "pasta",
    "constraints": [
        "gluten-free",
        "vegan",
        "under 500 calories per serving",
        "high protein (>15g per serving)",
        "no coconut",
        "taste must be rated 7/10 or higher",
    ],
}


# --- Helpers ---
def _extract_json(s: str) -> Dict[str, Any]:
    """
    Extract a JSON object from a string that may contain prose and/or code fences.
    Falls back to the largest {...} block if present.
    """
    s = s.strip()
    # Remove code fences if present
    if s.startswith("```"):
        s = s.strip("`")
        # if fenced with language (e.g., ```json)
        if "\n" in s:
            s = s.split("\n", 1)[1]
    # Try direct parse
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        pass
    # Fallback: find first '{' and last '}' to slice probable JSON
    start = s.find("{")
    end = s.rfind("}")
    if start != -1 and end != -1 and end > start:
        candidate = s[start : end + 1]
        return json.loads(candidate)
    raise ValueError("Could not parse JSON from model output.")


# --- Agents ---
class RecipeCreatorAgent:
    """
    Creates or revises a recipe to meet constraints.
    Returns a markdown-formatted recipe string with required sections.
    """

    def __init__(self, model: str = "gpt-4", temperature: float = 0.7):
        self.model = model
        self.temperature = temperature

    def create_recipe(
        self, recipe_request: Dict[str, Any], feedback: Optional[str] = None
    ) -> str:
        base_dish = recipe_request.get("base_dish", "dish")
        constraints = recipe_request.get("constraints", [])

        system = (
            "You are an expert chef AND a registered dietitian. "
            "Your job is to produce delicious recipes that strictly meet dietary constraints. "
            "When constraints conflict, prioritize dietary restrictions first (e.g., vegan, gluten-free, no-coconut), "
            "then nutrition targets (calories, protein), then taste. "
            "Always include accurate, conservative nutrition estimates per serving."
        )

        user = f"""
Create or revise a recipe based on the following request.

Base dish: {base_dish}

Constraints (must meet ALL):
{os.linesep.join(f"- {c}" for c in constraints)}

{'Evaluator feedback to incorporate:\n' + feedback if feedback else ''}

Formatting REQUIREMENTS (strict):
- Title on first line with a leading '# '
- Then sections in this exact order with markdown headings:
  ## Ingredients
  (bullet list; specify amounts; include gluten-free substitutions explicitly if relevant)
  ## Instructions
  (numbered steps; concise, actionable)
  ## Estimated Nutrition (per serving)
  - Servings: <integer>
  - Calories: <integer> kcal
  - Protein: <integer> g
  - Carbs: <integer> g
  - Fat: <integer> g
  (These values are REQUIRED. Use conservative, realistic estimates.)
  ## Taste Notes
  (1 short paragraph on flavor profile and texture)

Additional guidance:
- Avoid coconut in any form if 'no coconut' is a constraint.
- For high protein vegan recipes, consider legumes, tofu/tempeh, seitan (if gluten-free is NOT required), high-protein pasta, pea protein, or nuts (if permitted).
- For gluten-free, ensure pasta/grains and any sauces are explicitly gluten-free.
- Keep calories under the stated limit per serving if required; use roasting, acids, herbs, umami (miso/tamari if GF), and mindful fats.
"""

        resp = client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=[{"role": "system", "content": system}, {"role": "user", "content": user}],
        )
        return resp.choices[0].message.content.strip()


class NutritionEvaluatorAgent:
    """
    Evaluates a recipe string vs constraints.
    Returns a structured dict:
    {
      "results": [{"constraint": str, "pass": bool, "comments": str}, ...],
      "taste_rating": int (1-10),
      "overall_pass": bool,
      "unmet_constraints": [str],
      "feedback": str
    }
    """

    def __init__(self, model: str = "gpt-4", temperature: float = 0.0):
        self.model = model
        self.temperature = temperature

    def evaluate(self, recipe_markdown: str, constraints: List[str]) -> Dict[str, Any]:
        system = (
            "You are a certified nutritionist and compliance auditor for recipes. "
            "Be strict and literal: only mark a constraint as PASS if the recipe clearly satisfies it. "
            "If nutrition values are missing or unclear, FAIL the related constraint. "
            "If 'gluten-free' or 'vegan' are constraints, verify all ingredients and implied ingredients. "
            "If 'no coconut' appears anywhere, FAIL. "
            "If calories are required under a threshold, use the declared 'Estimated Nutrition (per serving)'. "
            "If protein target is specified, use the declared protein per serving."
        )

        schema = {
            "type": "object",
            "properties": {
                "results": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "constraint": {"type": "string"},
                            "pass": {"type": "boolean"},
                            "comments": {"type": "string"},
                        },
                        "required": ["constraint", "pass", "comments"],
                    },
                },
                "taste_rating": {"type": "integer", "minimum": 1, "maximum": 10},
                "overall_pass": {"type": "boolean"},
                "unmet_constraints": {"type": "array", "items": {"type": "string"}},
                "feedback": {"type": "string"},
            },
            "required": ["results", "taste_rating", "overall_pass", "unmet_constraints", "feedback"],
        }

        user = f"""
Evaluate the following recipe against the constraints. Return ONLY JSON conforming to this schema:
{json.dumps(schema, indent=2)}

Constraints:
{os.linesep.join(f"- {c}" for c in constraints)}

Recipe (markdown):
\"\"\"{recipe_markdown}\"\"\"

Evaluation instructions:
1) For each constraint, set pass=true/false with a brief comment.
2) Provide a taste_rating from 1–10 based on 'Taste Notes' and the structure of the recipe.
3) overall_pass must be true ONLY if all constraints pass (including taste threshold if present).
4) unmet_constraints should list the exact constraints that failed.
5) feedback: provide short, actionable guidance to FIX the failed constraints (ingredients, techniques, specific targets).
"""

        resp = client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=[{"role": "system", "content": system}, {"role": "user", "content": user}],
        )
        content = resp.choices[0].message.content
        return _extract_json(content)


# --- Orchestration ---
def optimize_recipe(
    recipe_request: Dict[str, Any], max_retries: int = MAX_RETRIES
) -> Tuple[str, Dict[str, Any]]:
    creator = RecipeCreatorAgent()
    evaluator = NutritionEvaluatorAgent()

    constraints = recipe_request.get("constraints", [])
    feedback: Optional[str] = None
    final_recipe = ""
    last_eval: Dict[str, Any] = {}

    for attempt in range(1, max_retries + 1):
        print(f"\n--- Attempt #{attempt} ---")
        final_recipe = creator.create_recipe(recipe_request, feedback=feedback)
        print("\n📝 Generated Recipe:\n")
        print(final_recipe)

        last_eval = evaluator.evaluate(final_recipe, constraints)
        print("\n🧪 Evaluation:\n")
        for r in last_eval.get("results", []):
            status = "PASS" if r["pass"] else "FAIL"
            print(f"- {r['constraint']}: {status} — {r['comments']}")
        print(f"🎯 Taste rating: {last_eval.get('taste_rating')}/10")
        print(f"✅ Overall pass: {last_eval.get('overall_pass')}")
        if last_eval.get("unmet_constraints"):
            print("❌ Unmet constraints:", ", ".join(last_eval["unmet_constraints"]))
        print("💡 Feedback:", last_eval.get("feedback", "").strip())

        if last_eval.get("overall_pass"):
            print("\n🎉 Success: All constraints satisfied.\n")
            break
        else:
            feedback = last_eval.get("feedback", "Revise to better satisfy all constraints.")

    return final_recipe, last_eval


# --- Entry Point (example run) ---
if __name__ == "__main__":
    recipe, eval_report = optimize_recipe(RECIPE_REQUEST)
    print("\n=== FINAL RESULT ===")
    print("\nRecipe:\n")
    print(recipe)
    print("\nConstraint Report:\n")
    print(json.dumps(eval_report, indent=2))


--- Attempt #1 ---

📝 Generated Recipe:

# Gluten-Free, Vegan High-Protein Pasta

## Ingredients
- 2 cups of gluten-free, high-protein chickpea pasta
- 1 tablespoon olive oil
- 4 cloves garlic, minced
- 1 red bell pepper, diced
- 2 cups of fresh spinach
- 1 cup of canned chickpeas, drained and rinsed
- 1 cup of cherry tomatoes, halved
- 1/4 cup of nutritional yeast
- Salt and pepper to taste
- Fresh basil leaves for garnish

## Instructions
1. Cook the gluten-free, high-protein chickpea pasta according to package instructions. Drain and set aside.
2. In a large skillet, heat the olive oil over medium heat. Add the minced garlic and cook until fragrant.
3. Add the diced bell pepper to the skillet and sauté until softened.
4. Add the spinach, chickpeas, and cherry tomatoes to the skillet. Cook until the spinach is wilted and the tomatoes are soft.
5. Add the cooked pasta to the skillet and stir in the nutritional yeast. Season with salt and pepper.
6. Cook for an additional 2 minutes, s